# 🎨 MASUKA - AI Generation Platform

**Train LoRAs and generate ultra-realistic images with NO restrictions**

---

## 🚀 One-Click Setup

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU (or A100 for faster training)
2. **Run the cell below**: Everything is automated!
   - Downloads latest code
   - Configures environment and database
   - Runs migrations automatically
   - Installs dependencies
   - Launches backend & frontend
3. **Access GUI**: Click the URL when it appears

---

## ✨ Features

- **LoRA Training**: Train character LoRAs with your images (5-15 min)
- **Image Generation**: Create unlimited images with trained characters
- **Dataset Management**: Upload, organize, and caption images
- **Auto-Migration**: Database schema automatically updated
- **Persistent Storage**: Everything saved to Google Drive

---

## 🔄 To Update & Restart

Just stop and re-run the cell below. It will:
- Pull latest code from GitHub
- Apply any new database migrations
- Restart everything fresh

**No manual steps needed!**

---

In [ ]:
# 🚀 MASUKA - Complete Setup & Launch
# This single cell handles everything automatically

import os
import sys
import time
import subprocess
from pathlib import Path

# ============================================================================
# STEP 0: MOUNT GOOGLE DRIVE
# ============================================================================
print("=" * 70)
print("🎨 MASUKA - AI Generation Platform")
print("=" * 70)
print("\n📂 Mounting Google Drive...")

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✓ Google Drive mounted")
except Exception as e:
    print(f"⚠️  Drive mount failed: {e}")
    print("  Continuing without persistent storage...")

# ============================================================================
# STEP 1: CLEANUP & DOWNLOAD
# ============================================================================
print("\n📦 STEP 1/3: Cleaning and downloading latest version...")
print("-" * 70)

print("🧹 Removing previous installations...")
!rm -rf /content/CharForgex /content/ai-gen /content/masuka 2>/dev/null || true
!find /content -type d -name __pycache__ -exec rm -rf {} + 2>/dev/null || true
print("✓ Cleanup complete")

print("\n📥 Downloading from GitHub...")
# Check if directory exists
if Path('/content/ai-gen').exists():
    print("  Repository already exists, pulling updates...")
    os.chdir('/content/ai-gen')
    !git pull origin main
else:
    print("  Cloning repository...")
    !git clone https://github.com/SamuelD27/ai-gen.git /content/ai-gen
    
# Verify download succeeded
if not Path('/content/ai-gen').exists():
    print("❌ ERROR: Failed to download repository!")
    print("   Please check your internet connection and try again.")
    raise SystemExit("Repository download failed")

os.chdir('/content/ai-gen')
print("✓ Latest code downloaded")

# ============================================================================
# STEP 2: RUN SETUP SCRIPT
# ============================================================================
print("\n📚 STEP 2/3: Installing dependencies and launching...")
print("-" * 70)
print("⚙️  Running setup script (this takes 10-15 minutes on first run)...")
print("  The setup script will:")
print("    • Configure environment and database")
print("    • Run database migrations automatically")
print("    • Install packages and download models")
print("    • Launch backend and frontend services")
print("    • Create secure tunnel for access")
print("")

# Run setup script - it handles EVERYTHING including migrations
!python masuka_colab_setup.py

# ============================================================================
# STEP 3: FINAL VALIDATION
# ============================================================================
print("\n💚 STEP 3/3: Final validation...")
print("-" * 70)

# Give services time to stabilize
print("⏳ Waiting for services to stabilize...")
time.sleep(5)

# Final database check
db_check_script = '''
import sys
sys.path.insert(0, "charforge-gui/backend")
from app.core.database import SessionLocal, engine
from sqlalchemy import inspect, text

try:
    db = SessionLocal()
    
    # Test basic query
    result = db.execute(text("SELECT COUNT(*) FROM training_sessions"))
    count = result.fetchone()[0]
    
    # Check for error_message column
    inspector = inspect(engine)
    columns = [col["name"] for col in inspector.get_columns("training_sessions")]
    
    if "error_message" in columns:
        print(f"✓ Database OK: {count} training sessions, error_message column present")
    else:
        print(f"⚠️  Database working but error_message column missing")
        print(f"   (Found {count} sessions)")
    
    db.close()
except Exception as e:
    print(f"❌ Database error: {e}")
    sys.exit(1)
'''

with open('/tmp/check_db.py', 'w') as f:
    f.write(db_check_script)

try:
    os.chdir('/content/ai-gen')
    result = !python /tmp/check_db.py 2>&1
    for line in result:
        print(f"  {line}")
except Exception as e:
    print(f"⚠️  Database check failed: {e}")

# ============================================================================
# ACCESS INFO
# ============================================================================
print("\n" + "=" * 70)
print("✨ MASUKA IS READY!")
print("=" * 70)

print("\n🌐 Access your GUI:")
time.sleep(2)

# Try to get ngrok URL
try:
    ngrok_check = !curl -s http://localhost:4040/api/tunnels 2>/dev/null || echo ""
    if ngrok_check and len(ngrok_check) > 0 and 'public_url' in str(ngrok_check):
        import json
        try:
            tunnels = json.loads(' '.join(ngrok_check))
            if 'tunnels' in tunnels and len(tunnels['tunnels']) > 0:
                url = tunnels['tunnels'][0]['public_url']
                print(f"\n  🔗 {url}")
                print(f"\n  📋 Copy and paste this URL in your browser")
        except:
            print("\n  ⏳ Tunnel still initializing - check setup output above for URL")
    else:
        print("\n  ⏳ Tunnel still initializing - check setup output above for URL")
except:
    print("\n  📝 Check the setup output above for your ngrok URL")

print("\n📊 System Status:")
print("  ✓ Google Drive: Mounted for persistent storage")
print("  ✓ Code: Latest version from GitHub")
print("  ✓ Database: Migrated and validated")
print("  ✓ Backend: Running on port 8000")
print("  ✓ Frontend: Running with ngrok tunnel")

print("\n💡 Quick Tips:")
print("  • Upload images in the Media tab")
print("  • Create datasets with trigger words")
print("  • Train LoRAs (takes 5-15 minutes)")
print("  • Generate unlimited images!")

print("\n🔄 To restart with latest code:")
print("  1. Stop this cell (⏹️ button)")
print("  2. Run: %cd /content/ai-gen && git pull")
print("  3. Re-run this cell")

print("\n" + "=" * 70)
print("🎉 Ready to create! Have fun!")
print("=" * 70)
print("\n⚠️  Keep this cell running - services will stop if interrupted")